#Bootsrapping

El bootstrapping (o bootstrap) es un método de remuestreo propuesto por Bradley Efron en 1979. Se utiliza para aproximar la distribución en el muestreo de un estadístico. Se usa frecuentemente para aproximar el sesgo o la varianza de un análisis estadístico, así como para construir intervalos de confianza o realizar contrastes de hipótesis sobre parámetros de interés.

In [48]:
import pandas as pd
import numpy as np
import random

In [52]:
# Muestra aleatoria de 10,0000 personas que tienen una edad promedio de 34 años
data = np.random.normal(loc = 34, size = 10000)
data

array([36.35852545, 34.10154052, 34.18987128, ..., 34.66133112,
       33.11899861, 34.0679494 ])

In [53]:
# Promedio de edad de la muestra anterior
data.mean()

34.000253459909935

Iniciamos el procedimiento para calcular el promedio de edad a partir de bootstraping

In [54]:
# Creando 40 muestras de tamaño 5 para estimar el promedio
promedio = []
for i in range(40):
  muestra = random.sample(data.tolist(), 5)
  prom = np.mean(muestra)
  promedio.append(prom)

In [55]:
np.mean(promedio)

33.93251865633372

#Kolmogorov - Smirnov

A menudo, en estadística, necesitamos comprender si una muestra determinada proviene de una distribución específica, más comúnmente la distribución normal (o gaussiana). Para ello disponemos de los llamados test de normalidad, como el de Shapiro-Wilk, el de Anderson-Darling o el de Kolmogorov-Smirnov.

Todos ellos miden la probabilidad de que una muestra provenga de una distribución normal, con un valor p relacionado para respaldar esta medida.

La prueba de Kolmogorov-Smirnov, sin embargo, va un paso más allá y nos permite comparar dos muestras y nos dice la probabilidad de que ambas provengan de la misma distribución. Esta prueba es muy útil para evaluar modelos de regresión y clasificación.

In [ ]:
from scipy import stats

In [ ]:
def cdf(sample, x, sort = False):
    # Sorts the sample, if unsorted
    if sort:
        sample.sort()
    # Counts how many observations are below x
    cdf = sum(sample <= x)
    # Divides by the total number of observations
    cdf = cdf / len(sample)
    return cdf

**Ejemplo 1: Prueba de Kolmogorov-Smirnov de una muestra**

La prueba KS de "1 muestra" usa en gran medida para verificar si una muestra se distribuye normalmente. *Evaluación de la normalidad*

In [ ]:
def ks_norm(sample):
    # Sorts the sample
    sample.sort()
    # Evaluates the KS statistic
    D_ks = [] # KS Statistic list
    for x in sample:
        cdf_normal = stats.norm.cdf(x = x, loc = 0, scale = 1)
        cdf_sample = cdf(sample = sample, x  = x)
        D_ks.append(abs(cdf_normal - cdf_sample))
    ks_stat = max(D_ks)
    # Calculates the P-Value based on the two-sided test
    # The P-Value comes from the KS Distribution Survival Function (SF = 1-CDF)
    p_value = stats.kstwo.sf(ks_stat, len(sample))
    return {"ks_stat": ks_stat, "p_value" : p_value}

In [ ]:
# Create random samples
norm_a = np.random.normal(loc = 0, scale = 1, size = 500)
norm_b = np.random.normal(loc = 0.1, scale = 1, size = 500)
norm_c = np.random.normal(loc = 3, scale = 1, size = 500)
f_a = np.random.f(dfnum = 5, dfden  = 10, size = 500)

In [ ]:
def standardize(sample):
  return (sample-sample.mean())/sample.std()

In [ ]:
# Performs the KS normality test in the samples
ks_norm_a = ks_norm(standardize(norm_a))
ks_norm_b = ks_norm(standardize(norm_b))
ks_norm_c = ks_norm(standardize(norm_c))
ks_f_a = ks_norm(standardize(f_a))
# Prints the result
print(f"norm_a: ks = {ks_norm_a['ks_stat']:.4f} (p-value = {ks_norm_a['p_value']:.3e}, is normal = {ks_norm_a['p_value'] > 0.05})")
print(f"norm_b: ks = {ks_norm_b['ks_stat']:.4f} (p-value = {ks_norm_b['p_value']:.3e}, is normal = {ks_norm_b['p_value'] > 0.05})")
print(f"norm_c: ks = {ks_norm_c['ks_stat']:.4f} (p-value = {ks_norm_c['p_value']:.3e}, is normal = {ks_norm_c['p_value'] > 0.05})")
print(f"f_a: ks = {ks_f_a['ks_stat']:.4f} (p-value = {ks_f_a['p_value']:.3e}, is normal = {ks_f_a['p_value'] > 0.05})")

norm_a: ks = 0.0326 (p-value = 6.498e-01, is normal = True)
norm_b: ks = 0.0254 (p-value = 8.949e-01, is normal = True)
norm_c: ks = 0.0301 (p-value = 7.441e-01, is normal = True)
f_a: ks = 0.1752 (p-value = 6.765e-14, is normal = False)


Comparamos el valor p con la significación. Si p<0.05 rechazamos la hipótesis nula y asumimos que la muestra no proviene de una distribución normal, como ocurre con f_a. Las otras tres muestras se consideran normales, como se esperaba.

Las muestras **norm_a** y **norm_b** provienen de una distribución normal y son realmente similares. La muestra **norm_c** también proviene de una distribución normal, pero con una media más alta. La **f_a** muestra proviene de una distribución F.

*Utilizando Scipy*

In [ ]:
# Evaluates the KS test con Scipy
ks_norm_a = stats.ks_1samp(x = standardize(norm_a), cdf = stats.norm.cdf)
ks_norm_b = stats.ks_1samp(x = standardize(norm_b), cdf = stats.norm.cdf)
ks_norm_c = stats.ks_1samp(x = standardize(norm_c), cdf = stats.norm.cdf)
ks_f_a = stats.ks_1samp(x = standardize(f_a), cdf = stats.norm.cdf)
# Prints the result
print(ks_norm_a)
print(ks_norm_b)
print(ks_norm_c)
print(ks_f_a)

KstestResult(statistic=0.03261445027727561, pvalue=0.6497811985101853)
KstestResult(statistic=0.027416753410563294, pvalue=0.8363021034453832)
KstestResult(statistic=0.030087575192675964, pvalue=0.7441252601492239)
KstestResult(statistic=0.1752158314197968, pvalue=6.765041816963118e-14)


**Ejemplo 2: Prueba de Kolmogorov-Smirnov de dos muestras**

In [ ]:
def ks_2samp(sample1, sample2):
    # Gets all observations
    observations = np.concatenate((sample1, sample2))
    observations.sort()
    # Sorts the samples
    sample1.sort()
    sample2.sort()
    # Evaluates the KS statistic
    D_ks = [] # KS Statistic list
    for x in observations:
        cdf_sample1 = cdf(sample = sample1, x  = x)
        cdf_sample2 = cdf(sample = sample2, x  = x)
        D_ks.append(abs(cdf_sample1 - cdf_sample2))
    ks_stat = max(D_ks)
    # Calculates the P-Value based on the two-sided test
    # The P-Value comes from the KS Distribution Survival Function (SF = 1-CDF)
    m, n = float(len(sample1)), float(len(sample2))
    en = m * n / (m + n)
    p_value = stats.kstwo.sf(ks_stat, np.round(en))
    return {"ks_stat": ks_stat, "p_value" : p_value}

In [ ]:
# Evaluates all possible combinations.
# We want to know if the distributions are identical, so we cannot standardize them
sets = [norm_a, norm_b, norm_c, f_a]
names = ['norm_a', 'norm_b', 'norm_c', 'f_a']
ks_scores = {}
for _ in range(len(names)):
    name1 = names.pop(0)
    sample1 = sets.pop(0)
    for name2, sample2 in zip(names, sets):
        key1 = name1 + "_" +  name2
        key2 = name2 + "_" +  name1
        ks = ks_2samp(sample1, sample2) # con Scipy: stats.ks_2samp(sample1, sample2)
        ks_scores[key1] = ks
        ks_scores[key2] = ks
# Prints the results
print(f"norm_a vs norm_b: ks = {ks_scores['norm_a_norm_b']['ks_stat']:.4f} (p-value = {ks_scores['norm_a_norm_b']['p_value']:.3e}, are equal = {ks_scores['norm_a_norm_b']['p_value'] > 0.05})")
print(f"norm_a vs norm_c: ks = {ks_scores['norm_a_norm_c']['ks_stat']:.4f} (p-value = {ks_scores['norm_a_norm_c']['p_value']:.3e}, are equal = {ks_scores['norm_a_norm_c']['p_value'] > 0.05})")
print(f"norm_a vs f_a: ks = {ks_scores['norm_a_f_a']['ks_stat']:.4f} (p-value = {ks_scores['norm_a_f_a']['p_value']:.3e}, are equal = {ks_scores['norm_a_f_a']['p_value'] > 0.05})")
print(f"norm_b vs norm_c: ks = {ks_scores['norm_b_norm_c']['ks_stat']:.4f} (p-value = {ks_scores['norm_b_norm_c']['p_value']:.3e}, are equal = {ks_scores['norm_b_norm_c']['p_value'] > 0.05})")
print(f"norm_b vs f_a: ks = {ks_scores['norm_b_f_a']['ks_stat']:.4f} (p-value = {ks_scores['norm_b_f_a']['p_value']:.3e}, are equal = {ks_scores['norm_b_f_a']['p_value'] > 0.05})")
print(f"norm_c vs f_a: ks = {ks_scores['norm_c_f_a']['ks_stat']:.4f} (p-value = {ks_scores['norm_c_f_a']['p_value']:.3e}, are equal = {ks_scores['norm_c_f_a']['p_value'] > 0.05})")

norm_a vs norm_b: ks = 0.0800 (p-value = 7.715e-02, are equal = True)
norm_a vs norm_c: ks = 0.8740 (p-value = 4.234e-225, are equal = False)
norm_a vs f_a: ks = 0.5560 (p-value = 2.993e-73, are equal = False)
norm_b vs norm_c: ks = 0.8640 (p-value = 1.169e-216, are equal = False)
norm_b vs f_a: ks = 0.5020 (p-value = 8.147e-59, are equal = False)
norm_c vs f_a: ks = 0.7140 (p-value = 2.591e-129, are equal = False)


Como era de esperar, solo las muestras **norm_a** y **norm_b** se pueden muestrear de la misma distribución para una significancia del 5%. No podemos considerar que las distribuciones de todos los demás pares sean iguales.

##**Bibliografía**

*   https://deepnote.com/@a_mas/Bootstrapping-en-Python-9ad885fd-c261-475e-b2b7-6461fd65490a
*   https://towardsdatascience.com/comparing-sample-distributions-with-the-kolmogorov-smirnov-ks-test-a2292ad6fee5

